# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\2imm10\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [3]:
#create co-occurrence matrix
np.set_printoptions(threshold=np.nan)
coMatrix=np.zeros((V, V), np.int8)

def processSentence(sentence):
    # Run over the sentence from all words as a co-occurrence
    for w1 in range(0, len(sentence)):
        word1 = sentence[w1]
        
        # Set window size
        window = []
        for x in range(1, window_size_corpus+1):
            window.append(w1+x)
            window.append(w1-x)
        
        # Compare w1 to every w2 in the window size 
        for w2 in window:
            if w2 >= 0 and w2 < len(sentence):
                word2 = sentence[w2]
                if(word1 != word2):
                    coMatrix[word1][word2] += 1

for sentence in corpus:
    processSentence(sentence)

# Normalize data to  be a percentage (value between 0 and 1, depending on the maximum value in the entire matrix).
divMatrix = np.zeros((V,V))
divMatrix.fill(coMatrix.max())
coMatrix = np.divide(coMatrix, divMatrix)

In [4]:
#find cosine similarity to Alice, Dinah and Rabbit
dictionary = tokenizer.word_index
alice = dictionary['alice']
rabbit = dictionary['rabbit']
dinah = dictionary['dinah']

print("Cosine similarity between Alice and Rabbit:")
print(cosine_similarity(coMatrix[alice].reshape(1, -1), coMatrix[rabbit].reshape(1, -1)))
print("\nCosine similarity between Alice and Dinah:")
print(cosine_similarity(coMatrix[alice].reshape(1, -1), coMatrix[dinah].reshape(1, -1)))
print("\nCosine similarity between Dinah and Rabbit:")
print(cosine_similarity(coMatrix[dinah].reshape(1, -1), coMatrix[rabbit].reshape(1, -1)))

Cosine similarity between Alice and Rabbit:
[[0.47890931]]

Cosine similarity between Alice and Dinah:
[[0.39360011]]

Cosine similarity between Dinah and Rabbit:
[[0.29862324]]


In [5]:
#find the 5 closest words to Alice

# Had to reshape data, but then I got an error that 1183 is not in the list. 1183 shouldn't be in the list, so why does
# it want to check it?
# This only happens with coMatrixPerc. TODO
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or 
# array.reshape(1, -1) if it contains a single sample.

#Find nearest neighbors
neighbors = nn(n_neighbors=6)
neighbors.fit(coMatrix) 
similars = (neighbors.kneighbors(coMatrix[alice].reshape(1, -1))[1])[0]

#translate ids back into words:
for word in similars:
    print(list(dictionary.keys())[list(dictionary.values()).index(word)])

alice
her
that
herself
for
on


Discussion of the drawbacks:

In [426]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension
f = open('vectors_co_occurrence.txt',"w")

# Write all the words in the file
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = coMatrix[:,1:]
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [7]:
#reopen your file as follows
co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow


In [242]:
#create CBOW model


In [243]:
#define loss function


In [1]:
#train model


In [ ]:
#prepare data for Skipgram


In [ ]:
#create Skipgram model


In [ ]:
#define loss function for Skipgram


In [ ]:
#train Skipgram model


In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [ ]:
#create Skipgram with additional dense layer


In [ ]:
#define loss function for Skipgram + dense


In [ ]:
#train model for Skipgram + dense


In [ ]:
#Implement your own analogy function


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings